In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

In [66]:
import itertools
from collections import Counter

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

from src.utils.logger import logger
from src.utils.io_utils import save_model

In [4]:
dataset = 'electronics'

### Import edges

In [68]:
df = pd.read_csv('../data/{}_edges_train.csv'.format(dataset))
val = pd.read_csv('../data/{}_edges_val.csv'.format(dataset))

In [69]:
train_set = set(df['product1'].tolist() + df['product2'].tolist())

In [71]:
val_set = set(val['product1'].tolist() + val['product2'].tolist())

In [73]:
train_set.intersection(val_set)

{'b005v154ww',
 'b005axit14',
 'b008lftcak',
 'b000rfpbqq',
 'b0054l1som',
 'b00c02yjmg',
 'b005mfnxb6',
 'b00hpm1g8q',
 'b001t8bdoe',
 'b00466leh2',
 'b002pnxu6g',
 'b00dlmh4je',
 'b00hvuegvq',
 'b000snbsmi',
 'b005laihzy',
 'b00eusfsdm',
 'b00dmy2bps',
 'b003x009pu',
 'b007o9j54m',
 'b00iixxbny',
 'b00a9trdwa',
 'b00cpr2234',
 'b00e6czpjo',
 'b0026sy76m',
 'b00c1t8p1o',
 'b004ehz7hq',
 'b0038qs514',
 'b0043x3l1k',
 'b002ovviuo',
 'b00frb912u',
 'b00ecu4g0o',
 'b003m6agsa',
 'b0087ucq66',
 'b006y43kvw',
 'b00iio39zi',
 'b0082xn57w',
 'b000a3i9yq',
 'b0088d0wze',
 'b00kh87u30',
 'b0038w0nou',
 'b0055wrvw8',
 'b003ns4hqy',
 'b00d8yiu7u',
 'b00cxtaste',
 'b000p5rusk',
 'b005hp3oyw',
 'b003dvtw1q',
 'b007ntjo8u',
 'b00auk8y58',
 'b000i7ox9y',
 'b006ts7s16',
 'b0038omeqi',
 'b00au6ipcy',
 'b00969ss4g',
 'b00gprcdfm',
 'b004msrq26',
 'b00i05efo4',
 'b007bxzypa',
 'b003es5zsw',
 'b008md3rny',
 'b000m57ck4',
 'b0041tqs5m',
 'b00ar2x8sm',
 'b001q9ev9k',
 'b007i7dgbs',
 'b007d6y58m',
 'b00e1rvt

### Calculate association

In [46]:
# Reverse product1 and product2 to get both directions
df_reversed = df.copy()
product1 = df['product1']
product2 = df['product2']
df_reversed['product1'], df_reversed['product2'] = product2, product1
df_reversed.head()

,product1,product2,weight
0,b004lwjf36,b0002khzho,1.0
1,b0052ogtbs,b004nhcgjo,1.0
2,b0042x9l6a,b0007u00xk,2.0
3,b0066130ko,b002oncd06,1.0
4,b007lqn8uu,b003coc7iy,2.0


In [47]:
df = pd.concat([df, df_reversed])

In [48]:
product_total_weight = df.groupby('product1').agg(total_weight=('weight', 'sum')).reset_index()

In [49]:
df = pd.merge(df, product_total_weight, how='left', left_on='product1', right_on='product1')

In [51]:
# Divide weight by total weight to get relative weight
df['relative_weight'] = df['weight'] / df['total_weight']

In [64]:
assoc_dict = df.groupby('product1').apply(lambda x: dict(zip(x['product2'], x['relative_weight']))).to_dict()

In [67]:
save_model(assoc_dict, '../model/assoc_dict_elec')

2019-12-13 12:24:22,575 - Model saved to ../model/assoc_dict_elec


In [74]:
len(assoc_dict)

360456

In [75]:
len(train_set)

360456

In [76]:
len(val_set)

275427